# Finetuning Ranker

Gratidão ao código base fornecido pelo colega de curso Thiago Soares Laitz

# Installs

In [ ]:
pip install bitsandbytes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install accelerate -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install transformers[torch]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Infra

## Função de verificação de memória

In [ ]:
from psutil import virtual_memory

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Jun 24 16:43:32 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
def mostra_memoria(lista_mem=['cpu']):
  """
  Esta função exibe informações de memória da CPU e/ou GPU, conforme parâmetros fornecidos.

  Parâmetros:
  -----------
  lista_mem : list, opcional
      Lista com strings 'cpu' e/ou 'gpu'.
      'cpu' - exibe informações de memória da CPU.
      'gpu' - exibe informações de memória da GPU (se disponível).
      O valor padrão é ['cpu'].

  Saída:
  -------
  A função não retorna nada, apenas exibe as informações na tela.

  Exemplo de uso:
  ---------------
  Para exibir informações de memória da CPU:
      mostra_memoria(['cpu'])

  Para exibir informações de memória da CPU e GPU:
      mostra_memoria(['cpu', 'gpu'])

  Autor: Marcus Vinícius Borela de Castro

  """
  if 'cpu' in lista_mem:
    vm = virtual_memory()
    ram={}
    ram['total']=round(vm.total / 1e9,2)
    ram['available']=round(virtual_memory().available / 1e9,2)
    # ram['percent']=round(virtual_memory().percent / 1e9,2)
    ram['used']=round(virtual_memory().used / 1e9,2)
    ram['free']=round(virtual_memory().free / 1e9,2)
    ram['active']=round(virtual_memory().active / 1e9,2)
    ram['inactive']=round(virtual_memory().inactive / 1e9,2)
    ram['buffers']=round(virtual_memory().buffers / 1e9,2)
    ram['cached']=round(virtual_memory().cached/1e9 ,2)
    print(f"Your runtime RAM in gb: \n total {ram['total']}\n available {ram['available']}\n used {ram['used']}\n free {ram['free']}\n cached {ram['cached']}\n buffers {ram['buffers']}")
    print('/nGPU')
    gpu_info = !nvidia-smi
  if 'gpu' in lista_mem:
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
      print('Not connected to a GPU')
    else:
      print(gpu_info)


In [ ]:
mostra_memoria(['cpu','gpu'])

Your runtime RAM in gb: 
 total 13.61
 available 12.45
 used 0.82
 free 7.01
 cached 5.72
 buffers 0.06
/nGPU
Sat Jun 24 16:43:32 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+--

## Imports

In [ ]:
import os

In [ ]:
from transformers import Trainer
import numpy as np
import torch


In [ ]:
import pandas as pd

In [ ]:
from dataclasses import dataclass, field

In [ ]:
from datasets import Dataset

In [ ]:
import json
from transformers import (
    AutoTokenizer,
    AutoConfig,
    MT5Tokenizer,
    AutoModelForSeq2SeqLM,
    AutoModelForSequenceClassification,
    Trainer,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorWithPadding,
    DataCollatorForSeq2Seq,
)
from tqdm.auto import tqdm
from datasets import Dataset
from dataclasses import dataclass, field


## Preparando para debug e display

In [ ]:
def config_display():
  """
  Esta função configura as opções de display do Pandas.
  """

  # Configurando formato saída Pandas
  # define o número máximo de colunas que serão exibidas
  pd.options.display.max_columns = None

  # define a largura máxima de uma linha
  pd.options.display.width = 1000

  # define o número máximo de linhas que serão exibidas
  pd.options.display.max_rows = 100

  # define o número máximo de caracteres por coluna
  pd.options.display.max_colwidth = 50

  # se deve exibir o número de linhas e colunas de um DataFrame.
  pd.options.display.show_dimensions = True

  # número de dígitos após a vírgula decimal a serem exibidos para floats.
  pd.options.display.precision = 7


In [ ]:
def config_debug():
  """
  Esta função configura as opções de debug do PyTorch e dos pacotes
  transformers e datasets.
  """

  # Define opções de impressão de tensores para o modo científico
  torch.set_printoptions(sci_mode=True)
  """
    Significa que valores muito grandes ou muito pequenos são mostrados em notação científica.
    Por exemplo, em vez de imprimir o número 0.0000012345 como 0.0000012345,
    ele seria impresso como 1.2345e-06. Isso é útil em situações em que os valores dos tensores
    envolvidos nas operações são muito grandes ou pequenos, e a notação científica permite
    uma melhor compreensão dos números envolvidos.
  """

  # Habilita detecção de anomalias no autograd do PyTorch
  torch.autograd.set_detect_anomaly(True)
  """
    Permite identificar operações que podem causar problemas de estabilidade numérica,
    como gradientes explodindo ou desaparecendo. Quando essa opção é ativada,
    o PyTorch verifica se há operações que geram valores NaN ou infinitos nos tensores
    envolvidos no cálculo do gradiente. Se for detectado um valor anômalo, o PyTorch
    interrompe a execução e gera uma exceção, permitindo que o erro seja corrigido
    antes que se torne um problema maior.

    É importante notar que a detecção de anomalias pode ter um impacto significativo
    no desempenho, especialmente em modelos grandes e complexos. Por esse motivo,
    ela deve ser usada com cautela e apenas para depuração.
  """

  # Configura variável de ambiente para habilitar a execução síncrona (bloqueante) das chamadas da API do CUDA.
  os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
  """
    o Python aguarda o término da execução de uma chamada da API do CUDA antes de executar a próxima chamada.
    Isso é útil para depurar erros no código que envolve operações na GPU, pois permite que o erro seja capturado
    no momento em que ocorre, e não depois de uma sequência de operações que pode tornar a origem do erro mais difícil de determinar.
    No entanto, é importante lembrar que esse modo de execução é significativamente mais lento do que a execução assíncrona,
    que é o comportamento padrão do CUDA. Por isso, é recomendado utilizar esse comando apenas em situações de depuração
    e removê-lo após a solução do problema.
  """

  # Define o nível de verbosity do pacote transformers para info
  # transformers.utils.logging.set_verbosity_info()


  """
    Define o nível de detalhamento das mensagens de log geradas pela biblioteca Hugging Face Transformers
    para o nível info. Isso significa que a biblioteca irá imprimir mensagens de log informativas sobre
    o andamento da execução, tais como tempo de execução, tamanho de batches, etc.

    Essas informações podem ser úteis para entender o que está acontecendo durante a execução da tarefa
    e auxiliar no processo de debug. É importante notar que, em alguns casos, a quantidade de informações
    geradas pode ser muito grande, o que pode afetar o desempenho do sistema e dificultar a visualização
    das informações relevantes. Por isso, é importante ajustar o nível de detalhamento de acordo com a
    necessidade de cada tarefa.

    Caso queira reduzir a quantidade de mensagens, comentar a linha acima e
      descomentar as duas linhas abaixo, para definir o nível de verbosity como error ou warning

    transformers.utils.logging.set_verbosity_error()
    transformers.utils.logging.set_verbosity_warning()
  """


  # Define o modo verbose do xmode, que é utilizado no debug
  # %xmode Verbose

  """
    Comando usado no Jupyter Notebook para controlar o modo de exibição das informações de exceções.
    O modo verbose é um modo detalhado que exibe informações adicionais ao imprimir as exceções.
    Ele inclui as informações de pilha de chamadas completa e valores de variáveis locais e globais
    no momento da exceção. Isso pode ser útil para depurar e encontrar a causa de exceções em seu código.
    Ao usar %xmode Verbose, as informações de exceção serão impressas com mais detalhes e informações adicionais serão incluídas.

    Caso queira desabilitar o modo verbose e utilizar o modo plain,
    comentar a linha acima e descomentar a linha abaixo:
    %xmode Plain
  """

  """
    Dica:
    1.  pdb (Python Debugger)
      Quando ocorre uma exceção em uma parte do código, o programa para a execução e exibe uma mensagem de erro
      com informações sobre a exceção, como a linha do código em que ocorreu o erro e o tipo da exceção.

      Se você estiver depurando o código e quiser examinar o estado das variáveis ​​e executar outras operações
      no momento em que a exceção ocorreu, pode usar o pdb (Python Debugger). Para isso, é preciso colocar o comando %debug
      logo após ocorrer a exceção. Isso fará com que o programa pare na linha em que ocorreu a exceção e abra o pdb,
      permitindo que você explore o estado das variáveis, examine a pilha de chamadas e execute outras operações para depurar o código.


    2. ipdb
      O ipdb é um depurador interativo para o Python que oferece recursos mais avançados do que o pdb,
      incluindo a capacidade de navegar pelo código fonte enquanto depura.

      Você pode começar a depurar seu código inserindo o comando ipdb.set_trace() em qualquer lugar do
      seu código onde deseja pausar a execução e começar a depurar. Quando a execução chegar nessa linha,
      o depurador entrará em ação, permitindo que você examine o estado atual do seu programa e execute
      comandos para investigar o comportamento.

      Durante a depuração, você pode usar comandos:
        next (para executar a próxima linha de código),
        step (para entrar em uma função chamada na próxima linha de código)
        continue (para continuar a execução normalmente até o próximo ponto de interrupção).

      Ao contrário do pdb, o ipdb é um depurador interativo que permite navegar pelo código fonte em que
      está trabalhando enquanto depura, permitindo que você inspecione variáveis, defina pontos de interrupção
      adicionais e até mesmo execute expressões Python no contexto do seu programa.
  """


In [ ]:
config_display()

In [ ]:
config_debug()

## Vinculando pasta do google drive para salvar dados

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
current_dir = os.getcwd()
print("Current directory:", current_dir)

Current directory: /content


## Constants

In [ ]:
TOKEN_FALSE = '▁não'
TOKEN_TRUE = '▁sim'

In [ ]:
MODEL_NAME = 'unicamp-dl/ptt5-base-pt-msmarco-100k-v2'
# 'unicamp-dl/mt5-3B-mmarco-en-pt'

# Carga dos dados

In [ ]:
PATH_LOCAL_DATA = '/content/drive/MyDrive/treinamento/202301_IA368DD/indir/data'

In [ ]:
# path_data = '/content/drive/MyDrive/treinamento/202301_IA368DD/indir/data/train_data_juris_tcu_index_bm25.csv'

PATH_TRAIN_DATA_ZIP = f"{PATH_LOCAL_DATA}/train_data_juris_tcu_index.zip"
PATH_TRAIN_DATA = f"{PATH_LOCAL_DATA}/train_data_juris_tcu_index.csv"

In [ ]:
os.path.exists(PATH_TRAIN_DATA)

True

In [ ]:
%%time
if not os.path.exists(PATH_TRAIN_DATA):
  import zipfile
  !wget https://github.com/marcusborela/ind-ir/raw/main/data/train_juris_tcu_index/train_data_juris_tcu_index.csv -O {PATH_TRAIN_DATA}

  # Extrair o arquivo zip
  with zipfile.ZipFile(PATH_TRAIN_DATA_ZIP, 'r') as zip_ref:
      zip_ref.extractall(PATH_LOCAL_DATA)

  # Listar os arquivos extraídos
  arquivos_extraidos = zip_ref.namelist()
  # Exibir os arquivos extraídos
  for arquivo in arquivos_extraidos:
      print(arquivo)
  print("File loaded")
else:
  print("File already there!")

File already there!
CPU times: user 851 µs, sys: 0 ns, total: 851 µs
Wall time: 1.04 ms


In [ ]:
df = pd.read_csv(PATH_TRAIN_DATA)

In [ ]:
df.shape

(282636, 6)

Verificando correção do arquivo!

In [ ]:
print(df.isnull().sum())

QUERY_ID          0
DOC_ID            0
RELEVANCE         0
TYPE_RELEVANCE    0
DOC_TEXT          0
QUERY_TEXT        0
Length: 6, dtype: int64


In [ ]:
df[['QUERY_TEXT','DOC_TEXT']].applymap(len).describe()

,QUERY_TEXT,DOC_TEXT
count,282636.0000000,282636.0000000
mean,314.4387445,971.9820688
std,159.8934908,671.0375228
min,41.0000000,86.0000000
25%,211.0000000,595.0000000
50%,286.0000000,812.0000000
75%,383.0000000,1162.0000000
max,4212.0000000,3739.0000000


Para cada positivo, tem 2 negativos

In [ ]:
df['RELEVANCE'].describe()

count    282636.0000000
mean          0.3333333
std           0.4714054
min           0.0000000
25%           0.0000000
50%           0.0000000
75%           1.0000000
max           1.0000000
Name: RELEVANCE, Length: 8, dtype: float64

# Treino

In [ ]:
# tokenizer = MT5Tokenizer.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

## Data selection

In [ ]:
df.shape

(282636, 6)

In [ ]:
# Se parcial
# df2 = df.head(20000)
# Se completa
df2 = df

In [ ]:
train_examples = {
    'query': df2['QUERY_TEXT'].tolist(),
    'text': df2['DOC_TEXT'].tolist(),
    'label': [TOKEN_FALSE if relevance == 0 else TOKEN_TRUE for relevance in df2['RELEVANCE']]
}


In [ ]:
train_examples_count = df2.shape[0]

In [ ]:
del df2

In [ ]:
train_examples['query'][0],train_examples['text'][0], train_examples['label'][0]

('SÚMULA TCU 1: Não se compreendem como vencimento, para efeito de concessão da pensão especial com fundamento na Lei nº 3.738, de 04/04/60, as vantagens previstas no art. 184 da Lei nº 1.711, de 28/10/52.',
 'O termo é "Pessoal".\nPessoal tem nota de escopo: "Designação genérica de todos os servidores ou funcionários civis pertencentes ao quadro de pessoal de um órgão ou entidade.".\nPessoal tem nota de escopo: "Tema agrupador para área de atuação do Controle Externo.".\nPessoal é uma generalização de: "Servidor público", "Funcionário público", "Pessoal civil", "Pessoal militar", "Pessoal temporário" e "Colaborador".\nPessoal tem termo relacionado: "Sisac", "Despesa com pessoal" e "HCAAF".\nPessoal tem tradução em espanhol: "Recursos humanos".\nPessoal tem tradução em inglês: "Personal" e "Human resources".',
 '▁sim')

In [ ]:
train_examples['query'][1],train_examples['text'][1], train_examples['label'][1]

('SÚMULA TCU 1: Não se compreendem como vencimento, para efeito de concessão da pensão especial com fundamento na Lei nº 3.738, de 04/04/60, as vantagens previstas no art. 184 da Lei nº 1.711, de 28/10/52.',
 'O termo é "Competência do TCU".\nCompetência do TCU tem definição: "São as seguintes as competências atribuídas ao TCU pela Constituição Federal de 1988: Art. 71. O controle externo, a cargo do Congresso Nacional, será exercido com o auxílio do Tribunal de Contas da União, ao qual compete:\r\nI - apreciar as contas prestadas anualmente pelo Presidente da República, mediante parecer prévio que deverá ser elaborado em sessenta dias a contar de seu recebimento;\r\nII - julgar as contas dos administradores e demais responsáveis por dinheiros, bens e valores públicos da administração direta e indireta, incluídas as fundações e sociedades instituídas e mantidas pelo Poder Público federal, e as contas daqueles que derem causa a perda, extravio ou outra irregularidade de que resulte prej

In [ ]:
print(train_examples_count)

282636


In [ ]:
def tokenize(batch):
    queries_documents = [f"Query: {query} Document: {text} Relevant:" for query, text in zip(batch["query"], batch["text"])]
    print(f"Chamado tokenize len(queries_documents): {len(queries_documents)}")
    tokenized = tokenizer(
        queries_documents,
        padding=True, # "max_length",
        truncation=True,
        # return_tensors="pt",
        max_length= 512
    )
    # tokenized["labels"] = [[label] for label in batch["label"]]
    # tokenized['label'] = [[token_false, token_true][int(pairs["label"][i])]
    tokenized["labels"] = tokenizer(batch['label'])['input_ids']
    # tokenized["labels"] = [tokenizer.get_vocab()[token] for token in batch['label']]
    # tokenized["labels"] = [token_id_true if label == 'true' else token_id_false for label in batch["label"]]
    return tokenized


In [ ]:
train_dataset = Dataset.from_dict(train_examples)

In [ ]:
%%time
# train_dataset.set_transform(tokenize)
train_dataset = train_dataset.map(
        tokenize,
        remove_columns=('query', 'text', 'label'),
        batched=True,
        desc='Tokenizing',
    )

Tokenizing:   0%|          | 0/282636 [00:00<?, ? examples/s]

Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_documents): 1000
Chamado tokenize len(queries_docum

In [ ]:
print(train_dataset[0])

{'input_ids': [2094, 540, 46, 140, 21799, 522, 14604, 89, 299, 165, 746, 10215, 852, 35, 5225, 41, 29, 6434, 6493, 3, 20, 1874, 4, 7170, 11, 18183, 918, 18, 11773, 22, 1090, 19, 31, 23152, 2617, 3, 4, 10837, 113, 2500, 113, 1869, 3, 42, 11563, 7866, 6, 19, 7904, 5, 472, 736, 11, 1090, 19, 18549, 2088, 3, 4, 687, 10392, 113, 4371, 5, 745, 797, 1806, 46, 28, 762, 21, 15, 3181, 1570, 329, 64, 4504, 77, 87, 2649, 4, 19414, 46, 15, 10444, 5219, 778, 21561, 4, 199, 30, 11121, 52, 3461, 4174, 9349, 37, 3155, 4, 1979, 4, 16, 3326, 52, 3593, 5, 64, 4504, 77, 87, 2649, 4, 19414, 46, 15, 2990, 186, 10727, 428, 20, 138, 4, 2704, 10, 15943, 1489, 413, 157, 5, 64, 4504, 77, 21, 17, 1841, 754, 4, 46, 15, 10810, 5574, 33, 534, 49, 15, 17532, 7069, 534, 49, 15, 3181, 1570, 329, 1760, 49, 15, 3181, 1570, 329, 721, 49, 15, 3181, 1570, 329, 15536, 27, 8, 15, 18973, 5800, 428, 64, 4504, 77, 87, 762, 5993, 46, 15, 6500, 6, 2090, 49, 15, 1270, 3037, 277, 18, 1979, 27, 8, 15, 14884, 89, 7354, 64, 4504, 77, 87

In [ ]:
print(train_dataset[1])

{'input_ids': [2094, 540, 46, 140, 21799, 522, 14604, 89, 299, 165, 746, 10215, 852, 35, 5225, 41, 29, 6434, 6493, 3, 20, 1874, 4, 7170, 11, 18183, 918, 18, 11773, 22, 1090, 19, 31, 23152, 2617, 3, 4, 10837, 113, 2500, 113, 1869, 3, 42, 11563, 7866, 6, 19, 7904, 5, 472, 736, 11, 1090, 19, 18549, 2088, 3, 4, 687, 10392, 113, 4371, 5, 745, 797, 1806, 46, 28, 762, 21, 15, 17664, 629, 1346, 10, 299, 165, 746, 64, 185, 4948, 1346, 10, 299, 165, 746, 87, 3269, 46, 15, 2925, 42, 614, 42, 8215, 6, 6083, 6, 37, 299, 165, 746, 50, 2854, 836, 4, 3887, 46, 3797, 5, 9510, 5, 28, 898, 8927, 3, 7, 758, 10, 2137, 381, 3, 1133, 15243, 18, 9, 7893, 10, 2330, 4, 3908, 6, 11, 842, 3, 37, 184, 8008, 46, 116, 78, 8187, 33, 42, 12558, 6469, 178, 4664, 54, 1562, 11, 651, 3, 6854, 9158, 19445, 13, 10166, 57, 9257, 12, 11566, 444, 7, 5065, 4, 45, 22771, 74, 500, 78, 18320, 42, 12558, 34, 20533, 8, 1809, 3996, 26, 1708, 6, 3, 3870, 8, 2123, 3205, 11, 1822, 3614, 8, 17004, 3, 12095, 42, 18692, 8, 10365, 16366, 6,

## setup

In [ ]:
!ls

drive  gdrive  sample_data


In [ ]:
 PATH_TRAIN_MODEL_LOCAL = "/content/drive/MyDrive/treinamento/202301_IA368DD/indir/train/monot5"

In [ ]:
training_args = Seq2SeqTrainingArguments(output_dir=PATH_TRAIN_MODEL_LOCAL)
# Needed to make the Trainer work with an on-the-fly transformation on the dataset
# training_args.remove_unused_columns = False
training_args.output_dir = PATH_TRAIN_MODEL_LOCAL
training_args.warmup_steps=200 # Alterar!
training_args.num_train_epochs=1.0 # Alterar!
training_args.logging_steps=200 # Alterar!
training_args.save_strategy="steps"
training_args.save_steps=200
training_args.save_total_limit=4
training_args.learning_rate=5e-5
training_args.per_device_train_batch_size=8 # Alterar!
training_args.gradient_accumulation_steps=4 # Alterar!
#training_args._n_gpu = 1
training_args.bf16 = True # se for usar a100, 3090, 4090 -> usar
training_args.evaluation_strategy = "no"
training_args.ignore_data_skip = True
training_args.do_eval = False
training_args.load_best_model_at_end = True
# training_args.optim='adamw_hf' #default
training_args.gradient_checkpointing = False # True
# se precisar economizar gpu
# training_args.optim='adamw_bnb_8bit'
# training_args.gradient_checkpointing = True
# training_args.report_to="none"

In [ ]:
print(training_args)

Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=True,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
generation_config=None,
generation_max_length=None,
generation_num_beams=None,
gradient_accumulation_steps=4,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend

In [ ]:
# from transformers.integrations import NeptuneCallback
# rastro_neptune = NeptuneRastroRun(hparam, parm_lista_tag= tag_contexto_rastro)
# neptune_callback = NeptuneCallback(run=rastro_neptune)

In [ ]:
%%time
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)


CPU times: user 2.8 s, sys: 1.19 s, total: 4 s
Wall time: 7.3 s


In [ ]:
trainer_cls = Seq2SeqTrainer
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [ ]:
# Limpa o cache da memória da GPU
# del trainer
torch.cuda.empty_cache()

## Train

In [ ]:
raise Exception('Parar aqui reinício!')

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
Exception: Parar aqui reinício!

In [ ]:
trainer = trainer_cls(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
%%time
train_metrics = trainer.train(resume_from_checkpoint=True)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1600,0.164600
1800,0.149500
2000,0.147500
2200,0.139800
2400,0.145200
2600,0.140400
2800,0.138300
3000,0.157000
3200,0.153800
3400,0.153500


Step,Training Loss
1600,0.164600
1800,0.149500
2000,0.147500
2200,0.139800
2400,0.145200
2600,0.140400
2800,0.138300
3000,0.157000
3200,0.153800
3400,0.153500


In [ ]:
raise Exception('Parar aqui reinício!')

In [ ]:
%%time
train_metrics = trainer.train()

In [ ]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

In [ ]:
%%time
train_metrics = trainer.train(resume_from_checkpoint=True)

antigos

In [ ]:
%%time
train_metrics = trainer.train(resume_from_checkpoint=True)

In [ ]:
print(train_metrics.metrics)

In [ ]:
trainer.save_model(training_args.output_dir)

In [ ]:
trainer.save_state()

In [ ]:
trainer.save_metrics('train', train_metrics.metrics)

In [ ]:
raise raise Exception('Parar aqui reinício!')

Teste inicio

In [ ]:
%%time
train_metrics = trainer.train()

In [ ]:
trainer = trainer_cls(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
train_metrics = trainer.train()